In [ ]:
%%capture
!pip install --upgrade tensorflow_datasets
!pip install --upgrade tensorflow
!pip install --upgrade nltk
!pip install --upgrade transformers
!pip install --upgrade rouge
!pip install --upgrade sentencepiece

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
%%capture
import tensorflow_datasets as tfds

In [ ]:
dataset = tfds.load('cnn_dailymail', split="test[:5%]") #Loading dataset via tensorflow_datasets

In [ ]:
#### Extract the documents and summaries
# Step 2: Preprocess
documents = [example['article'].numpy().decode() for example in dataset]
summaries = [example['highlights'].numpy().decode() for example in dataset]

In [ ]:
import random

for i in range(3):
    index = random.randint(0, len(documents))
    print(f"Document: \n--------\n{documents[index]}")
    print(f"Summary: \n--------\n{summaries[index]}\n")

Document: 
--------
In an unprecedented move, Russia has cancelled the release of a Hollywood thriller set in the Stalin era - claiming it distorts history and would air as the country celebrates its victory over Nazi Germany. The film, 'Child 44', starring Tom Hardy, Vincent Cassel and Gary Oldman, tells the story of a serial killer who targets children in the Stalin era. It is based on a novel by British writer Tom Rob Smith and was due to premiere in Russia today. Scroll down for video. Russian authorities have cancelled the release of Child 44, starring Gary Oldman (left) and Tom Hardy. The Hollywood blockbuster tells the story of a serial killer who targets children in Stalin era Russia. Russia's culture ministry said the film distorted history as it celebrated its victory over Nazi Germany. Russia's culture ministry said the film's distributors in Russia, Central Partnership, had agreed to withdraw a request for a distribution licence after ministry officials viewed the film. 'Fi

# Extractive Summarization with TF-IDF

In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow_datasets as tfds

#### TF-IDF for extractive summarization at sentence level

In [ ]:
def tfidf_extractive_summarization(document, top_n=3):
    # Tokenize the document into sentences
    sentences = nltk.sent_tokenize(document)

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the sentences
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Calculate the sentence scores
    sentence_scores = tfidf_matrix.sum(axis=1).tolist()

    # Get the indices of the top N sentences with the highest scores
    top_sentence_indices = sorted(range(len(sentence_scores)), key=lambda i: sentence_scores[i], reverse=True)[:top_n]

    # Sort the top sentences based on their original order in the document
    top_sentences = [sentences[i] for i in sorted(top_sentence_indices)]

    # Join the top sentences to form the summary
    summary = ' '.join(top_sentences)

    return summary

In [ ]:
example_index = random.randint(0, len(documents))  # selectin a doc to summarize
document = documents[example_index]
summary = summaries[example_index]

In [ ]:
tfidf_summary = tfidf_extractive_summarization(document)

In [ ]:
print("Original Document:")
print(document)
print()
print("Reference Summary:")
print(summary)
print()
print("TF-IDF Summary:")
print(tfidf_summary)
print()

Original Document:
Bending and rising in spectacular fashion, these stunning pictures capture the paddy fields of south east Asia and the arduous life of the farmers who cultivate them. In a photo album that spans over China, Thailand, Vietnam, Laos and Cambodia, extraordinary images portray the crop's full cycle from the primitive sowing of seeds to the distribution of millions of tonnes for consumption. The pictures were taken by professional photographer Scott Gable, 39, who spent four months travelling across the region documenting the labour and threadbare equipment used to harvest the carbohydrate-rich food. Scroll down for video. Majestic: A farmer wades through the mud with a stick as late morning rain falls on top of Dragonsbone terraces in Longsheng County, China. Rice is a staple food for more than one-half the world's population, but for many consumers, its origin remains somewhat of a mystery. The crop accounts for one fifth of all calories consumed by humans and 87 per ce

# Metrics Evaluation for Summarization

In [ ]:
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge

#### Loading the reference summaries and the generated summaries for TFIDF and BERTSum

In [ ]:
random_indices = [random.randint(0, len(documents)) for i in range(3)]

reference_summaries = [
    summaries[index] for index in random_indices
]

original_documents = [
    documents[index] for index in random_indices
]

tf_idf_generated_summaries = [
    tfidf_extractive_summarization(document) for document in original_documents
]


#### ROUGE Score

In [ ]:
#Calculate ROUGE-N, ROUGE-L, and ROUGE-S scores
def calculate_rouge_scores(reference_summaries, generated_summaries):
    rouge = Rouge()
    rouge_n_scores = []
    rouge_l_scores = []
    rouge_s_scores = []

    for reference, generated in zip(reference_summaries, generated_summaries):
        scores = rouge.get_scores(generated, reference)[0]

        # Extract ROUGE-N scores (unigram, bigram)
        rouge_n = [scores["rouge-1"]["f"], scores["rouge-2"]["f"]]
        rouge_n_scores.append(rouge_n)

        # Extract ROUGE-L scores
        rouge_l_scores.append(scores["rouge-l"]["f"])


    return rouge_n_scores, rouge_l_scores


In [ ]:
rouge_n_scores, rouge_l_scores = calculate_rouge_scores(reference_summaries, tf_idf_generated_summaries)

In [ ]:
print("ROUGE-N Scores:")
for n, scores in enumerate(rouge_n_scores, start=1):
    print(f"ROUGE-Summary-{n}: {scores}")
print()

print("ROUGE-L Scores:")
for i, score in enumerate(rouge_l_scores, start=1):
    print(f"ROUGE-L-Summary-{i}: {score}")
print()

ROUGE-N Scores:
ROUGE-Summary-1: [0.3232323191755943, 0.10810810433568717]
ROUGE-Summary-2: [0.3010752643727599, 0.15686274079200319]
ROUGE-Summary-3: [0.5072463718378492, 0.22222221725112012]

ROUGE-L Scores:
ROUGE-L-Summary-1: 0.30303029897357414
ROUGE-L-Summary-2: 0.21505375899641585
ROUGE-L-Summary-3: 0.49275361821466085



#### BLEU score

In [ ]:
def calculate_bleu_score(reference_summaries, generated_summaries):
    bleu_scores = []

    for reference, generated in zip(reference_summaries, generated_summaries):
        reference_tokens = reference.split()
        generated_tokens = generated.split()
        bleu = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(bleu)

    return bleu_scores

In [ ]:
bleu_scores = calculate_bleu_score(reference_summaries, tf_idf_generated_summaries)

In [ ]:
print("BLEU Scores:")
for i, score in enumerate(bleu_scores, start=1):
    print(f"BLEU-Sentence-{i}: {score}")

BLEU Scores:
BLEU-Sentence-1: 0.01729507217206357
BLEU-Sentence-2: 0.0934748138367647
BLEU-Sentence-3: 0.1076429664804074


#### METEOR score

In [ ]:
def calculate_meteor_score(reference_summaries, generated_summaries):
    meteor_scores = []

    for reference, generated in zip(reference_summaries, generated_summaries):
        meteor = meteor_score([reference.split()], generated.split())
        meteor_scores.append(meteor)

    return meteor_scores

In [ ]:
meteor_scores = calculate_meteor_score(reference_summaries, tf_idf_generated_summaries)

In [ ]:
print("METEOR Scores:")
for i, score in enumerate(meteor_scores, start=1):
    print(f"METEOR-Sentence-{i}: {score}")

METEOR Scores:
METEOR-Sentence-1: 0.34709901290961315
METEOR-Sentence-2: 0.3661019175203572
METEOR-Sentence-3: 0.3820668501024424


# Abstractive Summmarization

In [ ]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, BartTokenizer, PegasusTokenizer
from transformers import AutoTokenizer, PegasusForConditionalGeneration, pipeline, ProphetNetForConditionalGeneration, ProphetNetTokenizer
import tensorflow_datasets as tfds

#### BART for abstractive summarization

In [ ]:
def bart_abstractive_summarization(document):
    #  Load Model and Tokenize
    tokenizer = PreTrainedTokenizerFast.from_pretrained("facebook/bart-large-cnn")
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

    inputs = tokenizer(document, max_length=1024, return_tensors="pt")

    summary_ids = model.generate(inputs["input_ids"])
    # Decoding Text and return
    return tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [ ]:
document = documents[example_index]

#### Pegasus for abstractive summarization

In [ ]:
def pegasus_abstractive_summarization(document):

    model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail")
    tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")


    inputs = tokenizer(document, max_length=1024, return_tensors="pt")

    # Generate Summary
    summary_ids = model.generate(inputs["input_ids"])
    return tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

#### Generating the summaries using BART and Pegasus

In [ ]:
pegasus_summary = pegasus_abstractive_summarization(document)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (128) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
bart_summary = bart_abstractive_summarization(document)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (142) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the genera

In [ ]:
print("Original Document:")
print(document)
print()
print("Reference Summary:")
print(summary)
print()
print("BART Summary:")
print(bart_summary)
print()
print("Pegasus Summary:")
print(pegasus_summary)


Original Document:
Bending and rising in spectacular fashion, these stunning pictures capture the paddy fields of south east Asia and the arduous life of the farmers who cultivate them. In a photo album that spans over China, Thailand, Vietnam, Laos and Cambodia, extraordinary images portray the crop's full cycle from the primitive sowing of seeds to the distribution of millions of tonnes for consumption. The pictures were taken by professional photographer Scott Gable, 39, who spent four months travelling across the region documenting the labour and threadbare equipment used to harvest the carbohydrate-rich food. Scroll down for video. Majestic: A farmer wades through the mud with a stick as late morning rain falls on top of Dragonsbone terraces in Longsheng County, China. Rice is a staple food for more than one-half the world's population, but for many consumers, its origin remains somewhat of a mystery. The crop accounts for one fifth of all calories consumed by humans and 87 per ce

In [ ]:
bart_summaries = [
    bart_abstractive_summarization(document) for document in original_documents
]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Truncation was not explicitly act

In [ ]:
rouge_n_scores, rouge_l_scores = calculate_rouge_scores(reference_summaries, bart_summaries)
bleu_scores = calculate_bleu_score(reference_summaries, bart_summaries)
meteor_scores = calculate_meteor_score(reference_summaries, bart_summaries)

In [ ]:
print("ROUGE-N Scores:")
for n, scores in enumerate(rouge_n_scores, start=1):
    print(f"ROUGE-Summary-{n}: {scores}")
print()

print("ROUGE-L Scores:")
for i, score in enumerate(rouge_l_scores, start=1):
    print(f"ROUGE-L-Summary-{i}: {score}")
print()
print("BLEU Scores:")
for i, score in enumerate(bleu_scores, start=1):
    print(f"BLEU-Sentence-{i}: {score}")
print("METEOR Scores:")
for i, score in enumerate(meteor_scores, start=1):
    print(f"METEOR-Sentence-{i}: {score}")

ROUGE-N Scores:
ROUGE-Summary-1: [0.4057970966267591, 0.19178081718896617]
ROUGE-Summary-2: [0.4186046465521904, 0.23913043024574673]
ROUGE-Summary-3: [0.5352112626542352, 0.2643678111342318]

ROUGE-L Scores:
ROUGE-L-Summary-1: 0.3478260821340055
ROUGE-L-Summary-2: 0.395348832598702
ROUGE-L-Summary-3: 0.5211267556119817

BLEU Scores:
BLEU-Sentence-1: 0.12666081076387806
BLEU-Sentence-2: 0.1508349945785359
BLEU-Sentence-3: 0.2472029285786101
METEOR Scores:
METEOR-Sentence-1: 0.45636418966896974
METEOR-Sentence-2: 0.46620950258942556
METEOR-Sentence-3: 0.4679024125016793


In [ ]:
pegasus_summaries = [
    pegasus_abstractive_summarization(document) for document in original_documents
]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

In [ ]:
rouge_n_scores, rouge_l_scores = calculate_rouge_scores(reference_summaries, pegasus_summaries)
bleu_scores = calculate_bleu_score(reference_summaries, pegasus_summaries)
meteor_scores = calculate_meteor_score(reference_summaries, pegasus_summaries)

In [ ]:
print("ROUGE-N Scores:")
for n, scores in enumerate(rouge_n_scores, start=1):
    print(f"ROUGE-Summary-{n}: {scores}")
print()

print("ROUGE-L Scores:")
for i, score in enumerate(rouge_l_scores, start=1):
    print(f"ROUGE-L-Summary-{i}: {score}")
print()
print("BLEU Scores:")
for i, score in enumerate(bleu_scores, start=1):
    print(f"BLEU-Sentence-{i}: {score}")
print("METEOR Scores:")
for i, score in enumerate(meteor_scores, start=1):
    print(f"METEOR-Sentence-{i}: {score}")

ROUGE-N Scores:
ROUGE-Summary-1: [0.3055555508024692, 0.0821917760930759]
ROUGE-Summary-2: [0.3013698581272284, 0.13333332844088908]
ROUGE-Summary-3: [0.5179856065421046, 0.2721893441336088]

ROUGE-L Scores:
ROUGE-L-Summary-1: 0.3055555508024692
ROUGE-L-Summary-2: 0.27397259785325584
ROUGE-L-Summary-3: 0.5179856065421046

BLEU Scores:
BLEU-Sentence-1: 0.014011697931222915
BLEU-Sentence-2: 0.03330561328164072
BLEU-Sentence-3: 0.24350338602476607
METEOR Scores:
METEOR-Sentence-1: 0.24077669902912624
METEOR-Sentence-2: 0.302125123553695
METEOR-Sentence-3: 0.4557505506848245


In [ ]:
def prophetnet_abstractive_summarization(document):

    model = ProphetNetForConditionalGeneration.from_pretrained("microsoft/prophetnet-large-uncased-cnndm")
    tokenizer = AutoTokenizer.from_pretrained("microsoft/prophetnet-large-uncased-cnndm")


    inputs = tokenizer(document, max_length=1024, return_tensors="pt")

    # Generating Summary
    summary_ids = model.generate(inputs["input_ids"])
    return tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [ ]:
prophetnet_summary = prophetnet_abstractive_summarization(document)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
prophetnet_summary

'scott gable , 39 , spent four months travelling across the region documenting the labour and threadbare'

In [ ]:
prophetnet_summaries = [
    prophetnet_abstractive_summarization(document) for document in original_documents
]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

In [ ]:
rouge_n_scores, rouge_l_scores = calculate_rouge_scores(reference_summaries, prophetnet_summaries)
bleu_scores = calculate_bleu_score(reference_summaries, prophetnet_summaries)
meteor_scores = calculate_meteor_score(reference_summaries, prophetnet_summaries)

In [ ]:
print("ROUGE-N Scores:")
for n, scores in enumerate(rouge_n_scores, start=1):
    print(f"ROUGE-Summary-{n}: {scores}")
print()

print("ROUGE-L Scores:")
for i, score in enumerate(rouge_l_scores, start=1):
    print(f"ROUGE-L-Summary-{i}: {score}")
print()
print("BLEU Scores:")
for i, score in enumerate(bleu_scores, start=1):
    print(f"BLEU-Sentence-{i}: {score}")
print()
print("METEOR Scores:")
for i, score in enumerate(meteor_scores, start=1):
    print(f"METEOR-Sentence-{i}: {score}")

ROUGE-N Scores:
ROUGE-Summary-1: [0.0, 0.0]
ROUGE-Summary-2: [0.29787233593481216, 0.16666666222222234]
ROUGE-Summary-3: [0.12820512525969763, 0.06315789193573419]

ROUGE-L Scores:
ROUGE-L-Summary-1: 0.0
ROUGE-L-Summary-2: 0.29787233593481216
ROUGE-L-Summary-3: 0.12820512525969763

BLEU Scores:
BLEU-Sentence-1: 0
BLEU-Sentence-2: 0.033401409050084305
BLEU-Sentence-3: 0.004293055074527583

METEOR Scores:
METEOR-Sentence-1: 0.036101083032490974
METEOR-Sentence-2: 0.39178634416729646
METEOR-Sentence-3: 0.12550200803212855


In [ ]:
# Implementing T5 for abstractive summarization
def t5_abstractive_summarization(document):
    # T5 pipeline
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelWithLMHead

    tokenizer = AutoTokenizer.from_pretrained('T5-large')
    model = AutoModelWithLMHead.from_pretrained('T5-large', return_dict=True)
    # generating summary

    input_ids = tokenizer.encode(f"summarize: {document}", return_tensors='pt',
                                 max_length=512, truncation=True)
    summary_ids = model.generate(input_ids,
                min_length=20,
                max_length=80,
                num_beams=10,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True,
                no_repeat_ngram_size=2,
                use_cache=True,
                do_sample = True,
                temperature = 0.8,
                top_k = 50,
                top_p = 0.95)

    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary_text

In [ ]:
t5_summary = t5_abstractive_summarization(document)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1362: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
t5_summary

"photographer Scott gable spent four months travelling across south east Asia documenting the crop's full cycle. extraordinary images span china, Thailand, Vietnam, laos and Cambodia in a photo album that spans over six countries - including vietnam and thailand – as well as the united states of america (us)"

In [ ]:
#the original document and the summaries
print("Original Document:")
print(document)
print()
print("Reference Summary:")
print(summary)
print()
print("ProphetNET Summary:")
print(prophetnet_summary)
print()
print("T5 Summary:")
print(t5_summary)
print()


Original Document:
Bending and rising in spectacular fashion, these stunning pictures capture the paddy fields of south east Asia and the arduous life of the farmers who cultivate them. In a photo album that spans over China, Thailand, Vietnam, Laos and Cambodia, extraordinary images portray the crop's full cycle from the primitive sowing of seeds to the distribution of millions of tonnes for consumption. The pictures were taken by professional photographer Scott Gable, 39, who spent four months travelling across the region documenting the labour and threadbare equipment used to harvest the carbohydrate-rich food. Scroll down for video. Majestic: A farmer wades through the mud with a stick as late morning rain falls on top of Dragonsbone terraces in Longsheng County, China. Rice is a staple food for more than one-half the world's population, but for many consumers, its origin remains somewhat of a mystery. The crop accounts for one fifth of all calories consumed by humans and 87 per ce

In [ ]:
t5_summaries = [
    t5_abstractive_summarization(document) for document in original_documents
]

In [ ]:
rouge_n_scores, rouge_l_scores = calculate_rouge_scores(reference_summaries, t5_summaries)
bleu_scores = calculate_bleu_score(reference_summaries, t5_summaries)
meteor_scores = calculate_meteor_score(reference_summaries, t5_summaries)

In [ ]:
print("ROUGE-N Scores:")
for n, scores in enumerate(rouge_n_scores, start=1):
    print(f"ROUGE-Summary-{n}: {scores}")
print()

print("ROUGE-L Scores:")
for i, score in enumerate(rouge_l_scores, start=1):
    print(f"ROUGE-L-Summary-{i}: {score}")
print()
print("BLEU Scores:")
for i, score in enumerate(bleu_scores, start=1):
    print(f"BLEU-Sentence-{i}: {score}")
print()
print("METEOR Scores:")
for i, score in enumerate(meteor_scores, start=1):
    print(f"METEOR-Sentence-{i}: {score}")

ROUGE-N Scores:
ROUGE-Summary-1: [0.23529411269511738, 0.0]
ROUGE-Summary-2: [0.2857142807155456, 0.11764705384083066]
ROUGE-Summary-3: [0.4554455399117734, 0.3247863204003215]

ROUGE-L Scores:
ROUGE-L-Summary-1: 0.19607842642060758
ROUGE-L-Summary-2: 0.2857142807155456
ROUGE-L-Summary-3: 0.4356435597137536

BLEU Scores:
BLEU-Sentence-1: 0.009607695286097472
BLEU-Sentence-2: 0.040817873232580605
BLEU-Sentence-3: 0.17564334948553292

METEOR Scores:
METEOR-Sentence-1: 0.12280701754385967
METEOR-Sentence-2: 0.3906819424198251
METEOR-Sentence-3: 0.3151041666666666
